In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options

In [8]:
import time
import csv
import numpy as np
import pandas as pd
import requests
import re

from IPython.core.display import HTML

In [9]:
from login import sign_in

In [10]:
def sign_in(email, password):
#     this function automates login process on linkedin, 
#     provide email and password as strings
    # webdriver gets page
    driver.get("https://www.linkedin.com/")
    # pause for page to load
    time.sleep(3)
    # locate and send login email and password
    driver.find_element_by_id("session_key").send_keys(email)
    driver.find_element_by_id("session_password").send_keys(password)
    driver.find_element_by_class_name("sign-in-form__submit-button").click()
    return driver

In [29]:
def people_scrape(search_term, num_pages):
        
#                 function to automate search bar, search focus
#                 from your linkedin homepage, collecting data from
#                 search including name, location(secondary_deets), 
#                 headline(primary_deets) from condensed profiles returned from
#                 search results. requires string entry for search term
#                 and int input for number of pages (num_pages) 
#                 that you wish to scrape from results.

    # activate search bar cursor with click
    driver.find_element_by_css_selector("div#global-nav-search ").click()
    time.sleep(2)
    # send keyboard entry "div[id='oc-background-section']")for search terms
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(search_term)
    # send enter key to activate search
    driver.find_element_by_css_selector("input.search-global-typeahead__input").send_keys(Keys.RETURN)
    # wait for results to load
    driver.implicitly_wait(6)
    #w.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.reusable-search__entity-results-list')))
    element = driver.find_element_by_css_selector('ul.reusable-search__entity-results-list ')
    # scroll to element containing target(people_banner)  allowing ajax elements to load
    driver.execute_script("arguments[0].scrollIntoView();", element)
    driver.implicitly_wait(5)
   # locate banner under search type results
    people_banner = driver.find_element_by_link_text(f"See all people results")
    # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',people_banner)
    # pause to allow page to load
    driver.implicitly_wait(6)
    # starting with pagination page 1                       
    page_number = 1
    # get current url for page
    current_page_url = driver.current_url

    # set while loop to define pagination and data collection conditions
    while page_number <= num_pages:
        print("Processing page: " + str(page_number))
    # find all results on page
        links = driver.find_elements_by_css_selector("div.entity-result__content ")
   # pause for page load
        time.sleep(2)
    
    # iterate through results
        for l in links:
    # retrieve profile url
            title = l.find_element_by_css_selector("span.entity-result__title a.app-aware-link")
            profile_path = (str(title.get_attribute("pathname")))
    # add to urls list
            url_list.append(profile_path)
    # locating elements containing text needed
            details = l.find_elements_by_css_selector("div.linked-area")
    # the first element has the first three lines of text in the container
            deets = details[0]  
    # split text to assign elements appropriately
            text = deets.get_attribute('innerText').split('\n')
    # retrieve name/add to list
            name = text[0]
            names_list.append(name)
    # retrieve location/add to list   
            loc = text[-1]
            loc_list.append(loc)
     # retrieve headline/add to list
            headline = text[-2]
            headline_list.append(headline)
    # the second element selected contains the 'Current:' job text 
            current_job = details[1]
    # removing the 'Current:' string from text
            current_job = current_job.get_attribute('innerText').split(':')[1]
    # add current job to list
            current_job_list.append(current_job)
        time.sleep(3)
    # navigate using pagination function
        goto_next_page()
        page_number+=1
    # print to verify page during processing
        print(f"attempting to navigate to search results page {page_number}")
        time.sleep(5)
   
    # create dataframe with extracted information and save as csv file
    df = pd.DataFrame()                      
    df['name'] = names_list
    df['url'] = url_list
    df['current_job'] = current_job_list
    df['location'] = loc_list
    df['headline'] = headline_list
    # add complete url information for use in complete profile scraping
    for row in df:
        df['fetch'] = 'https://www.linkedin.com' + df.url + '/'
    df.to_csv(f'{search_term}.csv')
    # verify save
    print(f'{search_term}.csv saved')
    return df

In [30]:
def get_full_name():
    profile_header = driver.find_element_by_css_selector("ul.pv-top-card--list")
    # find name element from profile header portion
    header_details = profile_header.find_elements_by_tag_name("li")
    profile_name = header_details[0].get_attribute('innerText')
    return profile_name

def get_first_name():
    full_name = get_name()
    name = full_name.split(' ', 1)
    first_name = name[0]
    return first_name

In [51]:
#function to locate and interact with "next" button at bottom of search
def goto_next_page():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(7)
    driver.find_element_by_css_selector('div.artdeco-pagination.ember-view')
    go_to_next = driver.find_element_by_css_selector('button[aria-label="Next"]')
    driver.execute_script('arguments[0].click();',go_to_next)

In [45]:
def get_experience():
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
    time.sleep(5)
    # background container
    background = driver.find_element_by_css_selector(
        "div[id='oc-background-section']")
    #print("here" + str(background))
    driver.execute_script("arguments[0].scrollIntoView();", background)
    
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"section#experience-section")))
    # try/except clause used to locate element, not all people list jobs
    exp = background.find_element_by_css_selector(
            "section[id='experience-section']")
    history = exp.find_elements_by_css_selector('li.pv-entity__position-group-pager')
    details = history[0]
    # try/except clause used to locate element to avoid 'element not found' error ceasing from running
    try:
        pos = details.find_element_by_tag_name(
                'h3').get_attribute('outerText')
    except NoSuchElementException:
        pos = 'nan'

    try:
        company = details.find_element_by_tag_name(
                'p.pv-entity__secondary-title').get_attribute('innerText')
    except NoSuchElementException:
        company = 'nan'
        
    try:
        location = details.find_element_by_css_selector(
                'h4.pv-entity__location').get_attribute('innerText')
        location = location.split('\n', 1)[1]
    except NoSuchElementException:
        location = 'nan'
        
    try:
        dates = details.find_element_by_css_selector(
                "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]
    except NoSuchElementException:
            dates = 'nan'
    
    job_list.append(pos)
    company_list.append(company)
    date_list.append(dates)
    loc_list.append(location)

    if (len(history)) >=2:
        details = history[1]
        
        try:
            pos = details.find_element_by_tag_name(
                    'h3').get_attribute('outerText')
        except NoSuchElementException:
            pos = 'nan'
     
        try:
            company = details.find_element_by_tag_name(
                    'p.pv-entity__secondary-title').get_attribute('innerText')
        except NoSuchElementException:
            company = 'nan'
            
        try:
            location = details.find_element_by_css_selector(
                    'h4.pv-entity__location').get_attribute('innerText')
            location = location.split('\n', 1)[1]
        except NoSuchElementException:
            location = 'nan'
                #print(location)
        try:
            dates = details.find_element_by_css_selector(
                    "h4.pv-entity__date-range").get_attribute('innerText').split(' ', 2)[-1]

        except NoSuchElementException:
                dates = 'nan'

        if (len(history)) < 2:
            pos = 'nan'
            company  ='nan'
            location  ='nan'
            dates = 'nan'

        job_list2.append(pos)
        company_list2.append(company)
        date_list2.append(dates)
        loc_list2.append(location)
        
        
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
    time.sleep(2)

In [33]:
def get_education():
    # wait for element to proceed
    w = WebDriverWait(driver, 10)
    w.until(EC.presence_of_element_located((By.CSS_SELECTOR,"div.profile-detail")))
    # background container
    background = driver.find_element_by_css_selector(
        "div[id='oc-background-section']")
    #print("here" + str(background))
    driver.execute_script("arguments[0].scrollIntoView();", background)
    time.sleep(8)
    # locate education section
    education = background.find_element_by_css_selector(
            "section[id='education-section']")
 
    try:
        schools = education.find_elements_by_css_selector(
                "a[data-control-name='background_details_school']")
        school = schools[0]
        school_name = school.find_element_by_tag_name(
            'h3').get_attribute('innerText')
    except NoSuchElementException:
        school_name =  'nan'

    school_list.append(school_name)

In [34]:
def get_email():
    # scroll to top of profile where email is located
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    contact_info = driver.find_element_by_css_selector(
        'a[data-control-name="contact_see_more"]')
     # js function to click banner/button to see additional results under jobs, people, or posts                      
    driver.execute_script('arguments[0].click();',contact_info)
    time.sleep(3)
    try:
        container =  driver.find_elements_by_css_selector(
            'div.pv-contact-info__ci-container')
        email =  container[1].get_attribute('innerText')
     
    
    except (NoSuchElementException, IndexError):
        email = 'nan'
    
    email_list.append(email)
    close_popup = driver.find_element_by_css_selector(
            'button[aria-label="Dismiss" ]')
    close_popup.click()


In [35]:
def profile_connect(message):
    # scroll to top of profile to ensure elements can be found by webdriver
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    # locate & click 'Connect' button located at top of profile
    driver.find_element_by_class_name('pv-s-profile-actions--connect').click()
    action_bar = driver.find_element_by_css_selector("div.artdeco-modal__actionbar")
    # locate 'Add a note' button by class
    action_bar.find_element_by_class_name('mr1').click()

    time.sleep(3)
    # action to send text from message to input box
    message_input = driver.find_element_by_id('custom-message').send_keys(message)
    # send the message & connection request
    driver.find_element_by_class_name('ml1').click()

In [37]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

In [38]:
driver = sign_in('mathis.brennan@gmail.com', 'Shitballs1!')

In [39]:
name_list = []
email_list = []
position_list = []
company_list = []
location_list = []
dates_list = []
position_list2 = []
company_list2 = []
location_list2 = []
dates_list2 = []
school_list = []
school_list2 = []

In [40]:
loc_list = []
current_job_list = []
names_list = []
url_list = []
headline_list = []

#search_df = people_scrape('data analyst', 2)

In [49]:
def autoconnect(term, pages):
    loc_list = []
    current_job_list = []
    names_list = []
    url_list = []
    headline_list = []

    name_list = []
    email_list = []
    job_list = []
    company_list = []
    loc_list = []
    date_list = []
    job_list2 = []
    company_list2 = []
    loc_list2 = []
    date_list2 = []
    school_list = []
    
    search_df = people_scrape('data scientist ibm', 2)
    profile_urls = search_df['fetch']
    for url in profile_urls:
        driver.get(url)
        get_experience()
        get_education()
        get_email()
        
    detail_df = pd.DataFrame()
    detail_df['job1'] = job_list
    detail_df['company1'] = company_list
    detail_df['location1'] = loc_list
    detail_df['dates1'] = date_list
    detail_df['job2'] = job_list2
    detail_df['company2'] = company_list2
    detail_df['location2'] = loc_list2
    detail_df['dates2'] = date_list2
    detail_df['school'] = school_list
    detail_df['email'] = email_list
    df = pd.concat([search_df, detail_df], axis=1)
    df['first_name'] = df['name'].map(lambda x: x.split(' ')[0])
    df['personalized_message'] = (f"""Hi + {df['first_name']}, my name is Brennan, and I am a data scientist with a background in video.
                                  I see that you are a {df['job1']} at {df['company1']}. I am super interested in finding out more
                                  about what you do in your current position, so I just wanted to reach out, connect, and say hello!""")
    message_input = list(df['personalized_message'])
    counter = 0

    for url in profile_urls:
        driver.get(url)
        message = str(message_input[counter])
        #profile_connect(message)
        print(message)
        counter+=1
        print("You're now connected to:  " + df['name'] + " | " + df['company1'] + " | " + df['job1'] )

In [52]:
autoconnect('data engineer ibm', 2)

Processing page: 1
attempting to navigate to search results page 2
Processing page: 2
attempting to navigate to search results page 3
data scientist ibm.csv saved


NameError: name 'job_list' is not defined

In [ ]:
profile_urls = search_df['fetch']

In [120]:
%time

    name_list = []
    email_list = []
    job_list = []
    company_list = []
    loc_list = []
    date_list = []
    job_list2 = []
    company_list2 = []
    loc_list2 = []
    date_list2 = []
    school_list = []

    
def scrape_profile(profile_urls):
    profile_urls = search_df['fetch']
    for url in profile_urls:
        driver.get(url)
        get_experience()
        get_education()
        get_email()
        
    detail_df = pd.DataFrame()
    detail_df['job1'] = job_list
    detail_df['company1'] = company_list
    detail_df['location1'] = loc_list
    detail_df['dates1'] = date_list
    detail_df['job2'] = job_list2
    detail_df['company2'] = company_list2
    detail_df['location2'] = loc_list2
    detail_df['dates2'] = date_list2
    detail_df['school'] = school_list
    detail_df['email'] = email_list
    df = pd.concat([search_df, detail_df], axis=1)
    df['first_name'] = df['name'].map(lambda x: x.split(' ')[0])
    df['personalized_message'] = "Hi " + df['first_name'] 
                                    + ", my name is Brennan, and I am a data scientist looking "
                                    + "to segue into data engineering. I see that you are a "
                                    + df['job1'] + " at " + df['company1']+". I am super interested in finding out more"
                                    + " about what you do in your current position, so I just wanted to reach out, connect, and say hello!"
    message_input = list(df['personalized_message'])
    
    
def make_connection():
    counter = 0

    for url in profile_urls:
        driver.get(url)
        message = str(message_input[counter])
        profile_connect(message)


        counter+=1
    

Wall time: 0 ns


In [5]:
print("Hi " + df['first_name']
                                    + ", my name is Brennan, and I am a data scientist looking "
                                    + "to segue into data engineering. I see that you are a "
                                    + df['job1'] + " at " + df['company1']+". I am super interested in finding out more"
                                    + " about what you do in your current position, so I just wanted to reach out, connect, and say hello!")

NameError: name 'df' is not defined

In [116]:
get_info_df(profile_urls)

ValueError: Length of values (0) does not match length of index (2)

In [99]:
detail_df = pd.DataFrame()
detail_df['job1'] = job_list
detail_df['company1'] = company_list
detail_df['location1'] = loc_list
detail_df['dates1'] = date_list
detail_df['job2'] = job_list2
detail_df['company2'] = company_list2
detail_df['location2'] = loc_list2
detail_df['dates2'] = date_list2
detail_df['school'] = school_list

In [100]:
df = pd.concat([search_df, detail_df], axis=1)

In [101]:
df['first_name'] = df['name'].map(lambda x: x.split(' ')[0])

In [102]:
df

,name,url,current_job,location,headline,fetch,job1,company1,location1,dates1,job2,company2,location2,dates2,school,first_name
0,Murali Subramanian,/in/musubr0298,Data Engineer at Searce Inc,"Austin, TX",Data Engineer at Searce Inc. | 2x GCP | 2x AWS,https://www.linkedin.com/in/musubr0298/,Data Engineer,Slalom Build Full-time,"Chicago, Illinois, United States",2021 – Present,Machine Learning Engineer,BrainCo Internship,nan,2020 – Feb 2021,The George Washington University,Murali
1,Jianfang Chen,/in/jianfang-chen-2286a31b1,Data Engineering,United States,Data Engineer at Microsoft,https://www.linkedin.com/in/jianfang-chen-2286...,Data Engineer,Square,nan,2021 – Present,Company Name\nZoomSystems,nan,San Francisco Bay Area,2019 – Feb 2021,Hult International Business School,Jianfang
2,Vina Zhu,/in/vina-zhu-4986451a4,Data Engineer at Slalom Build,"Arlington, VA",Data Engineer,https://www.linkedin.com/in/vina-zhu-4986451a4/,Company Name\nFannie Mae,nan,"Reston, Virginia, United States",2020 – Present,Assitant Manager,Papa John's International,nan,2014 – Jul 2016,James Madison University,Vina
3,Palak Kaur,/in/palak-kaur,A Data Analyst and Engineer taking care of da...,Greater Seattle Area,Data Engineer at Square,https://www.linkedin.com/in/palak-kaur/,Data Engineer,Albertsons Companies,Greater Phoenix Area,2021 – Present,Company Name\nCiti,nan,"Irving, Texas, United States",2020 – Jan 2021,Southern Arkansas University,Palak
4,Alper Tosun,/in/alper-tosun-bba82467,Data Management Analyst at Fannie Mae - ...th...,Washington DC-Baltimore Area,Data Engineer at Fannie Mae,https://www.linkedin.com/in/alper-tosun-bba82467/,Data Engineer,Cognizant Full-time,nan,2020 – Present,Data Science Program,Thinkful,nan,2019 – Feb 2020,State University of New York at Binghamton,Alper
5,Venkat Nelluri,/in/vnelluri,Data Engineer at Albertsons Companies - I'm r...,Dallas-Fort Worth Metroplex,Data Engineer at Albertsons Companies,https://www.linkedin.com/in/vnelluri/,Data Engineer,Caris Life Sciences Full-time,"Phoenix, Arizona, United States",2021 – Present,Data Science Fellow,The Data Incubator Full-time,Online,2020 – Jun 2020,The Data Incubator,Venkat
6,Rachel Kiesling,/in/rachelkiesling,Data Science Program at Thinkful - - Learned ...,"Waltham, MA",Data Engineer at Cognizant,https://www.linkedin.com/in/rachelkiesling/,Data Engineer,COFENSE Full-time,"Leesburg, Virginia, United States",2020 – Present,Data Analyst,"Technomics, Inc. Full-time","Arlington, Virginia, United States",2019 – Feb 2020,Schreyer Honors College at The Pennsylvania St...,Rachel
7,Asfetaw Abera,/in/asfetaw-abera-54b0a2185,Data Science Fellow at The Data Incubator - C...,"Silver Spring, MD",Data Engineer at Caris Life Sciences,https://www.linkedin.com/in/asfetaw-abera-54b0...,Data Engineer,Verizon Contract,Remote,2020 – Present,"Intern, Analytics and Information Management",Amtrak Internship,Washington D.C. Metro Area,2019 – Apr 2020,George Mason University,Asfetaw
8,Elena Cadenas,/in/elena-cadenas-378285126,Data Engineer at COFENSE - Develop queries to...,"Chantilly, VA",Data Engineer at COFENSE,https://www.linkedin.com/in/elena-cadenas-3782...,Data Engineer,Capital One Full-time,Washington D.C. Metro Area,2020 – Present,Data Software Engineer,The Advisory Board Company,Washington D.C. Metro Area,2019 – Feb 2020,University of Maryland,Elena
9,Sruthi Pusuluri,/in/sruthipusuluri,Machine Learning Engineer Intern at NaturalText,"Fairfax, VA",Data Engineer at Verizon,https://www.linkedin.com/in/sruthipusuluri/,Data Engineer,Vanguard,"Malvern, Pennsylvania",2019 – Present,Company Name\nThe Johns Hopkins University,nan,"Baltimore, Maryland, United States",2019 – Jun 2019,The Johns Hopkins University,Sruthi


In [60]:
df['personalized_message'] = "Hi " + df['first_name']+", my name is Brennan, and I am a data scientist looking to segue into data engineering. I see that you are a "+df['job1']+ " at " +df['company1']+". I am super interested in finding out more about what you do in your current position, so I just wanted to reach out, connect, and say hello!"

In [86]:
message_input = list(df['personalized_message'])

In [88]:
counter = 0
for url in profile_urls:
    driver.get(url)
    message = str(message_input[counter])
    profile_connect(message)
    

    counter+=1
     
    

In [ ]:
driver.forward()
driver.back()

In [385]:
driver.quit()

In [71]:
profile_urls = df['fetch'][2:]

In [92]:
print(df.job1)

0                         Data Engineer
1                         Data Engineer
2                         Data Engineer
3                         Data Engineer
4              Company Name\nFannie Mae
5                         Data Engineer
6                         Data Engineer
7                         Data Engineer
8                         Data Engineer
9                         Data Engineer
10                        Data Engineer
11                        Data Engineer
12    Manager, BI/Dashboard Development
13                     Hadoop Developer
Name: job1, dtype: object
